<a href="https://colab.research.google.com/github/kawsarahmd/transformer-text-classifier/blob/main/text_multiclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers numpy pandas scikit-learn
!pip install datasets
!pip install tqdm

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

# Load the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data = pd.DataFrame({'text_data': newsgroups.data, 'label': newsgroups.target})

# Visualize newsgroup data object
entry_index = 0
print(f"Text:\n{newsgroups['data'][entry_index]}\n\n")
print(f"Label index: {newsgroups['target'][entry_index]}")
print(f"Label name: {newsgroups['target_names'][newsgroups['target'][entry_index]]}")

# Shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True)

# Split the dataset into training and validation sets (80:20 ratio)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Initialize BERT tokenizer using the pretrained 'bert-base-uncased' model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_seq_len = 128

def tokenize_data(data, tokenizer, max_seq_len):
    input_ids, attention_masks, labels = [], [], []

    # Iterate through each row in the dataset
    for index, row in tqdm(data.iterrows(), total=len(data)):
        # Tokenize the text using BERT's tokenizer with additional parameters
        encoded = tokenizer.encode_plus(
            row["text_data"],
            add_special_tokens=True,  # Add [CLS] and [SEP] tokens
            max_length=max_seq_len,  # Set max sequence length to 128
            padding="max_length",  # Pad shorter sequences to max_seq_len
            truncation=True,  # Truncate longer sequences to max_seq_len
            return_attention_mask=True,  # Return attention masks
        )

        # Append tokenized data to respective lists
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])
        labels.append(row["label"])

    # Convert lists to tensors
    return torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels)

# Tokenize both the training and validation data using the defined function
train_input_ids, train_attention_masks, train_labels = tokenize_data(train_data, tokenizer, max_seq_len)
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_data, tokenizer, max_seq_len)

Text:


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




Label index: 10
Label name: rec.sport.hockey


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 3770/3770 [00:13<00:00, 271.02it/s]


In [3]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create a TensorDataset object for the training set
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
# Use RandomSampler to shuffle the samples in the dataset
train_sampler = RandomSampler(train_dataset)
# Create DataLoader for the training set using dataset, sampler, and batch size
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Create a TensorDataset object for the validation set
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
# Use SequentialSampler to process the validation dataset sequentially
val_sampler = SequentialSampler(val_dataset)
# Create DataLoader for the validation set using dataset, sampler, and batch size
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

In [4]:
from transformers import BertForSequenceClassification, BertConfig
from torch.optim import AdamW

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=20,  # Number of labels (20) corresponds to the 20 newsgroups dataset
    output_attentions=False,  # Do not output attention weights
    output_hidden_states=False,  # Do not output hidden states
)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# 🚂 **BERT Model Training** 🚂

This final part involves fine-tuning the BERT model on the provided dataset and evaluating its performance. We define functions to train and evaluate the model after every epoch and calculate loss and accuracy metrics during training and validation, respectively. Furthermore, components such as optimizer and scheduler are introduced for efficient model training to help improve the results on each step. This section helps you understand the overall process of training BERT for a classification task and assessing the model's performance.

In [5]:
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report

num_epochs = 3
total_steps = len(train_dataloader) * num_epochs

# Create the optimizer and scheduler for fine-tuning the model
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0

    # Use a progress bar during training
    progress_bar = tqdm(dataloader, desc="Training", position=0, leave=True)

    # Iterate through each batch in a training epoch
    for batch in progress_bar:
        input_ids, attention_masks, labels = [t.to(device) for t in batch]

        # Zero out gradients before each backward pass
        optimizer.zero_grad()

        # Forward pass to compute the outputs and loss
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()

        # Perform a backward pass and update optimizer/scheduler steps
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_description(f"Training - Loss: {loss.item():.4f}")

    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0

    # Use a progress bar during evaluation
    progress_bar = tqdm(dataloader, desc="Evaluation", position=0, leave=True)

    # Iterate through each batch in a validation epoch
    for batch in progress_bar:
        input_ids, attention_masks, labels = [t.to(device) for t in batch]

        # Disable gradient calculations during evaluation
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks)

        logits = outputs[0].detach().cpu().numpy()
        label_ids = labels.cpu().numpy()

        # Calculate accuracy for the current batch
        batch_accuracy = accuracy_score(label_ids, logits.argmax(axis=-1))
        total_eval_accuracy += batch_accuracy

        progress_bar.set_description(f"Evaluation - Batch Accuracy: {batch_accuracy:.4f}")

    return total_eval_accuracy / len(dataloader)

# Train and evaluate the model for 'num_epochs' times
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    val_accuracy = evaluate(model, val_dataloader, device)

    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Loss: {train_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

Evaluation - Batch Accuracy: 0.6000: 100%|██████████| 236/236 [00:24<00:00,  9.45it/s]



Epoch 1/3
Loss: 1.5029 - Validation Accuracy: 0.7160


Evaluation - Batch Accuracy: 0.5000: 100%|██████████| 236/236 [00:25<00:00,  9.43it/s]



Epoch 2/3
Loss: 0.8028 - Validation Accuracy: 0.7383


Evaluation - Batch Accuracy: 0.5000: 100%|██████████| 236/236 [00:25<00:00,  9.43it/s]


Epoch 3/3
Loss: 0.5747 - Validation Accuracy: 0.7434


In [6]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

def get_predictions(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    # Use tqdm for a progress bar
    for batch in tqdm(dataloader, desc="Evaluating"):
        input_ids, attention_masks, labels = [t.to(device) for t in batch]

        # Disable gradient calculations during evaluation
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks)

        logits = outputs[0].detach().cpu().numpy()
        label_ids = labels.cpu().numpy()

        # Add batch logits and labels to the list of predictions and true labels
        predictions.extend(logits.argmax(axis=-1))
        true_labels.extend(label_ids)

    # Convert the lists to NumPy arrays
    return np.array(predictions), np.array(true_labels)

# Obtain the model's predictions and true labels for the validation dataset
predictions, true_labels = get_predictions(model, val_dataloader, device)

# Calculate the accuracy on the validation dataset
accuracy = accuracy_score(true_labels, predictions)

# Generate a classification report with more detailed performance metrics
report = classification_report(true_labels, predictions, digits=4)

# Print the accuracy and classification report
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Evaluating: 100%|██████████| 236/236 [00:25<00:00,  9.43it/s]

Validation Accuracy: 0.7438
Classification Report:
              precision    recall  f1-score   support

           0     0.4824    0.6260    0.5449       131
           1     0.7398    0.7214    0.7305       201
           2     0.6604    0.7035    0.6813       199
           3     0.6607    0.7014    0.6805       211
           4     0.7619    0.6919    0.7252       185
           5     0.8419    0.8265    0.8341       219
           6     0.8069    0.8316    0.8191       196
           7     0.5594    0.8466    0.6737       189
           8     0.8545    0.6980    0.7684       202
           9     0.9121    0.8691    0.8901       191
          10     0.9176    0.8836    0.9003       189
          11     0.8009    0.8317    0.8160       208
          12     0.7391    0.6700    0.7028       203
          13     0.8634    0.8850    0.8741       200
          14     0.8656    0.8173    0.8407       197
          15     0.6986    0.7487    0.7228       195
          16     0.6809    0.6

# 💾 **Save Model**

In [7]:
output_dir = "./model/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

In [19]:
def classify_news_article(model, tokenizer, device, text, max_len=128):
    # Tokenize the text using BERT tokenizer
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt",  # Return tensors
    )

    # Move tensors to device (GPU or CPU)
    input_ids, attention_mask = encoded["input_ids"].to(device), encoded["attention_mask"].to(device)

    # Predict the label using the trained model
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask)

    # Find the index of the class with the highest score
    predicted_label_index = logits[0].argmax(-1).item()

    return predicted_label_index


sample_news_article = "Linux is awesome"

# Get the predicted label index for the given sample article
predicted_label_index = classify_news_article(model, tokenizer, device, sample_news_article)

# Print the predicted label index and name
print(f"Predicted Label Index: {predicted_label_index}")
print(f"Predicted Class: {newsgroups.target_names[predicted_label_index]}")

Predicted Label Index: 2
Predicted Class: comp.os.ms-windows.misc


In [16]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']